# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import json

# Import API key
from api_keys import geoapify_key

import os

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lompoc,34.5134,-121.2834,288.66,88,100,6.94,US,1726284174
1,1,bohorodchany,48.7521,24.4929,287.80,96,100,1.53,UA,1726284177
2,2,tena,-1.3381,-77.4128,298.13,67,32,0.84,EC,1726284178
3,3,puerto natales,-21.1186,-68.2624,274.18,21,0,2.75,CL,1726284186
4,4,pathalgaon,5.3281,34.2634,300.18,44,34,2.37,SS,1726284196


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', #Longtitute and Latitude as the x and y coordinates
    geo=True, #Enable geographic plotting
    tiles='OSM', #Background map tiles
    size='Humidity', #Point size proportional to humidity
    color='Humidity', #Color scale based on humidity
    frame_width=700, #Map width
    frame_height=500, #Map height
    hover_cols=['City', 'Country', 'Humidity'] #Show additional info on hover
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
#Convert from kelvin to C
city_data_df['Max Temp (C)'] = city_data_df['Max Temp'] - 273.15
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp (C)'] >= 20) &
    (city_data_df['Max Temp (C)'] <= 30) &
    (city_data_df['Humidity'] <= 75)
]
# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (C)
2,2,tena,-1.3381,-77.4128,298.13,67,32,0.84,EC,1726284178,24.98
4,4,pathalgaon,5.3281,34.2634,300.18,44,34,2.37,SS,1726284196,27.03
8,8,saipan,-6.9919,30.5146,298.44,43,5,1.85,TZ,1726284204,25.29
17,17,juan griego,37.9572,-115.0102,295.11,14,0,3.19,US,1726284233,21.96
18,18,alekseyevskoye,15.8167,-2.6280,302.89,50,99,2.42,ML,1726284235,29.74


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,lompoc,US,34.5134,-121.2834,88,
1,bohorodchany,UA,48.7521,24.4929,96,
2,tena,EC,-1.3381,-77.4128,67,
3,puerto natales,CL,-21.1186,-68.2624,21,
4,pathalgaon,SS,5.3281,34.2634,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
# Your Geoapify API key
geoapify_key = "your_geoapify_key_here"

# Base URL for the Geoapify Places API (for searching places like hotels)
base_url = "https://api.geoapify.com/v2/places"

# Set the search radius (in meters)
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame for the current city
    lat = row['Lat']
    lng = row['Lng']

    # Update the params dictionary with the current city coordinates
    params = {
        "categories": "accommodation.hotel",  # Search for hotels
        "filter": f"circle:{lng},{lat},{radius}",  # Use circle filter with lat/lng and radius
        "bias": f"proximity:{lng},{lat}",  # Proximity bias
        "limit": 1,  # Get only the nearest hotel
        "apiKey": geoapify_key
    }
    # Make the API request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        try:
            # Convert the API response to JSON format
            name_address = response.json()

            # Extract the hotel name from the response if available
            if name_address['features']:
                hotel_name = name_address['features'][0]['properties']['name']
                # Update the 'Hotel Name' column in the DataFrame
                hotel_df.at[index, 'Hotel Name'] = hotel_name
            else:
                # If no hotel is found, set 'No Hotel Found'
                hotel_df.at[index, 'Hotel Name'] = "No Hotel Found"
        except (KeyError, IndexError) as e:
            print(f"Error parsing API response: {e}")
            hotel_df.at[index, 'Hotel Name'] = "No Hotel Found"
    else:
        print(f"Error: {response.status_code} for city {row['City']}")
        hotel_df.at[index, 'Hotel Name'] = "No Hotel Found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the updated DataFrame
hotel_df.head()

Starting hotel search
Error: 401 for city lompoc
lompoc - nearest hotel: No Hotel Found
Error: 401 for city bohorodchany
bohorodchany - nearest hotel: No Hotel Found
Error: 401 for city tena
tena - nearest hotel: No Hotel Found
Error: 401 for city puerto natales
puerto natales - nearest hotel: No Hotel Found
Error: 401 for city pathalgaon
pathalgaon - nearest hotel: No Hotel Found
Error: 401 for city adamstown
adamstown - nearest hotel: No Hotel Found
Error: 401 for city karema
karema - nearest hotel: No Hotel Found
Error: 401 for city tiksi
tiksi - nearest hotel: No Hotel Found
Error: 401 for city saipan
saipan - nearest hotel: No Hotel Found
Error: 401 for city georgetown
georgetown - nearest hotel: No Hotel Found
Error: 401 for city port moresby
port moresby - nearest hotel: No Hotel Found
Error: 401 for city troitsko-pechorsk
troitsko-pechorsk - nearest hotel: No Hotel Found
Error: 401 for city krasnokamsk
krasnokamsk - nearest hotel: No Hotel Found
Error: 401 for city yangor
yango

,City,Country,Lat,Lng,Humidity,Hotel Name
0,lompoc,US,34.5134,-121.2834,88,No Hotel Found
1,bohorodchany,UA,48.7521,24.4929,96,No Hotel Found
2,tena,EC,-1.3381,-77.4128,67,No Hotel Found
3,puerto natales,CL,-21.1186,-68.2624,21,No Hotel Found
4,pathalgaon,SS,5.3281,34.2634,44,No Hotel Found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
#%%capture --no-display
# Create the hover message including City, Country, Hotel Name, and Humidity
hotel_df['Hover Info'] = hotel_df.apply(
    lambda row: f"City: {row['City']}<br>Country: {row['Country']}<br>Hotel: {row['Hotel Name']}<br>Humidity: {row['Humidity']}%", axis=1)

# Plot the points on a map with the hover information
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,  # Activate geographic plot
    color='Humidity',  # Color by Humidity
    size=10,  # Size of the points
    hover_cols=['Hover Info'],  # Show the custom hover message
    tiles='CartoLight',  # Optional: Base map tiles
    frame_width=800,  # Adjust width
    frame_height=600  # Adjust height
)

# Show the map with hover info
hotel_map
# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hover Info)